#Question 2

In [1]:
from google.colab import files
uploaded = files.upload()

Saving mushrooms.csv to mushrooms.csv


In [2]:
import pandas as pd

mushrooms_data = pd.read_csv('/content/mushrooms.csv')

In [5]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Replace '?' with np.nan
mushrooms_data.replace('?', np.nan, inplace=True)

encoder = LabelEncoder()
mushrooms_encoded = mushrooms_data.apply(encoder.fit_transform)

X = mushrooms_encoded.drop('class', axis=1)
y = mushrooms_encoded['class']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=42)

len(X_train), len(X_val), len(X_test)


(5686, 1625, 813)

##Part 1

<div dir="rtl" style="font-family: BNazanin; text-align: justify;">
کنترل مقادیر ناقص یک مرحله حیاتی در پیش‌پردازش داده‌ها است، به ویژه در مجموعه داده‌ای مانند داده های این سوال که مقادیر ناقص می‌توانند تأثیر قابل توجهی بر عملکرد مدل‌های یادگیری ماشین داشته باشند. روش‌های متعددی برای کنترل داده‌های ناقص وجود دارند و انتخاب روش معمولاً بستگی به خصوصیات داده و درصد مقادیر ناقص دارد. در زیر، رویکرد ترجیحی خود را شرح میدهم:

رویکرد انتخابی:  (Imputation)

Imputation شامل جایگزین کردن مقادیر ناقص با مقادیر جایگزین است. روش Imputation می‌تواند بسته به خصوصیات داده متفاوت باشد. برای داده‌های گروهی، معمولاً از مقدار بیشترین تکرار در ستون  استفاده می‌شود. برای داده‌های عددی، گزینه‌هایی مانند میانگین یا میانه ستون مورد استفاده قرار می‌گیرند. Imputation به طور کلی ترجیح داده می‌شود زیرا به ما امکان می‌دهد تمام نقاط داده را حفظ کنیم که در مجموعه داده‌هایی با تعداد نمونه محدود بسیار مهم است.

 مراحل Imputation:

1. **شناسایی مقادیر ناقص**: تعیین کنید کدام ویژگی‌ها مقادیر ناقص دارند و میزان این مقادیر ناقص را تعیین کنید.
2. **انتخاب روش جانشینی**: بر اساس نوع داده (گروهی یا عددی) ستون‌ها، یک روش مناسب جانشینی را انتخاب کنید.
3. **اعمال جانشینی**: مقادیر ناقص را با مقادیر محاسبه شده (میانگین، میانه یا حالت) جایگزین کنید.
</div>

In [6]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_val = pd.DataFrame(imputer.transform(X_val), columns=X_val.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)


##Part 2

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

accuracies = {}
for depth in [4, 8, 16, 24, 32]:
    dt_classifier = DecisionTreeClassifier(max_depth=depth, criterion='gini', random_state=42)

    dt_classifier.fit(X_train, y_train)

    train_predictions = dt_classifier.predict(X_train)
    test_predictions = dt_classifier.predict(X_test)

    train_accuracy = accuracy_score(y_train, train_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)
    accuracies[depth] = {'Train Accuracy': train_accuracy, 'Test Accuracy': test_accuracy}

for depth in accuracies:
    print(f"Depth: {depth}, Train Accuracy: {accuracies[depth]['Train Accuracy']}, Test Accuracy: {accuracies[depth]['Test Accuracy']}")


Depth: 4, Train Accuracy: 0.9780161800914526, Test Accuracy: 0.985239852398524
Depth: 8, Train Accuracy: 1.0, Test Accuracy: 1.0
Depth: 16, Train Accuracy: 1.0, Test Accuracy: 1.0
Depth: 24, Train Accuracy: 1.0, Test Accuracy: 1.0
Depth: 32, Train Accuracy: 1.0, Test Accuracy: 1.0


##Part 3

In [11]:
from sklearn.metrics import precision_score
from itertools import product
from random import sample

def create_feature_subset(X, num_features):
    selected_features = sample(list(X.columns), num_features)
    return X[selected_features]

max_depths = [4, 8, 12, 16]
feature_numbers = [3, 5, 7]
n_trees = 7

results = []
best_model = None
best_accuracy = 0

feature_names = mushrooms_data.drop('class', axis=1).columns

X_train = pd.DataFrame(X_train, columns=feature_names)
X_test = pd.DataFrame(X_test, columns=feature_names)

for depth, feature_num in product(max_depths, feature_numbers):
    trees = []

    for _ in range(n_trees):
        X_subset = create_feature_subset(X_train, feature_num)
        tree = DecisionTreeClassifier(max_depth=depth, random_state=42)
        tree.fit(X_subset, y_train)
        trees.append((tree, X_subset.columns))

    train_predictions = []
    for i in range(len(X_train)):
        votes = [tree.predict(X_train.iloc[i][features].to_frame().T)[0] for tree, features in trees]
        train_predictions.append(max(set(votes), key=votes.count))

    accuracy = accuracy_score(y_train, train_predictions)
    results.append({'Depth': depth, 'Feature Number': feature_num, 'Accuracy': accuracy})

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = trees
# Variables to store the best hyperparameters
best_depth = None
best_feature_num = None

for result in results:
    print(f"Depth: {result['Depth']}, Feature Number: {result['Feature Number']}, Training Accuracy: {result['Accuracy']}")

    if result['Accuracy'] == best_accuracy:
        best_depth = result['Depth']
        best_feature_num = result['Feature Number']
        best_model = trees

print(f"Best hyperparameters - Depth: {best_depth}, Feature Number: {best_feature_num}, with Training Accuracy: {best_accuracy}")


Depth: 4, Feature Number: 3, Training Accuracy: 0.8877945831867745
Depth: 4, Feature Number: 5, Training Accuracy: 0.9423144565599718
Depth: 4, Feature Number: 7, Training Accuracy: 0.9857544846992613
Depth: 8, Feature Number: 3, Training Accuracy: 0.9458318677453394
Depth: 8, Feature Number: 5, Training Accuracy: 0.9880408019697503
Depth: 8, Feature Number: 7, Training Accuracy: 0.9989447766443897
Depth: 12, Feature Number: 3, Training Accuracy: 0.9423144565599718
Depth: 12, Feature Number: 5, Training Accuracy: 0.9961308476960957
Depth: 12, Feature Number: 7, Training Accuracy: 0.9956032360182905
Depth: 16, Feature Number: 3, Training Accuracy: 0.9393246570524094
Depth: 16, Feature Number: 5, Training Accuracy: 0.981885332395357
Depth: 16, Feature Number: 7, Training Accuracy: 1.0
Best hyperparameters - Depth: 16, Feature Number: 7, with Training Accuracy: 1.0


In [24]:
test_predictions = [max(set([tree.predict(X_test.iloc[i][features].to_frame().T)[0] for tree, features in best_model]), key=lambda x: [x].count(x)) for i in range(len(X_test))]

best_test_accuracy = accuracy_score(y_test, test_predictions)

encoder = LabelEncoder()
encoder.fit(mushrooms_data['class'])

positive_class_label = encoder.transform(['e'])[0]
best_precision = precision_score(y_test, test_predictions, pos_label=positive_class_label)


print(f"Best Model Test Accuracy: {best_test_accuracy}, Precision: {best_precision}")


Best Model Test Accuracy: 0.9163591635916359, Precision: 0.8586278586278586
